In [1]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime
import pandas as pd

In [2]:
ee.Initialize()

# VETTORI 1EGLF7Gh-5Z3Zt9pxdKF-jBDtxQ-DnpJzz9D1BGQJ
countries_gaul = ee.FeatureCollection('ft:1ZDEMjtnWm_smu7l_z3fx91BbxyCRzP2A3cEMrEiP')
africa_bbox = ee.Geometry.Rectangle(-15.64, -33.58, 59.06, 25.96)
filtered = countries_gaul.filterBounds(africa_bbox)

In [3]:
# AREA OF INTEREST
region = [[-25.0, -40.0], [65.0, -40.0], [65.0, 40.0], [-30.0, 40.0], [-30.0, -40.0]]
country_number = filtered.size().getInfo()

69

In [4]:
wpbm_stats = ee.Image('projects/fao-wapor/L1-WPbmY2015-sample');

In [5]:
def vizualize_image(wbpm,regione_plot,scala):    
    visparams = {"opacity":1,
             "min":0,
             "max":1.2,
             "palette":"bc170f,e97a1a,fff83a,9bff40,5cb326",
             "region": regione_plot,
             "scale" : scala}
    
    url_WPbm = wbpm.getThumbUrl(visparams)
    return url_WPbm

In [138]:
%matplotlib inline  
Image(url=vizualize_image(wpbm_stats,region,250))

#### Means

In [8]:
means = wpbm_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = 250,
)

#### Min Max

In [9]:
minMaxs = wpbm_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.minMax(),
    scale = 250
)

#### MinMax StdDev combined

In [10]:
reducers = ee.Reducer.minMax().combine(
    reducer2 = ee.Reducer.stdDev(),
    sharedInputs = True
);

minMaxStds = wpbm_stats.reduceRegions(
            collection = filtered,
            reducer = reducers,
            scale = 250,
);

## Export

In [102]:
features_m = means.getInfo()['features']
features_mms = minMaxStds.getInfo()['features']

In [119]:
df_m = pd.DataFrame(data=features_m[1:], columns = features_m[0])
df_mms = pd.DataFrame(data=features_mms[1:], columns = features_mms[0])

In [120]:
serie_m = df_m['properties'].apply(pd.Series)
serie_mms = df_mms['properties'].apply(pd.Series)

In [121]:
serie_m.columns

Index([u'bbox', u'bbox_flag', u'bbox_lod', u'claimed_by',
       u'claimed_by_ft_style', u'fid', u'gaul_code', u'geometry_vertex_count',
       u'import_notes', u'iso3', u'mean', u'name', u'region',
       u'region_ft_style', u'status', u'status_ft_style', u'subregion',
       u'subregion_ft_style'],
      dtype='object')

In [122]:
serie_mms.columns

Index([u'bbox', u'bbox_flag', u'bbox_lod', u'claimed_by',
       u'claimed_by_ft_style', u'fid', u'gaul_code', u'geometry_vertex_count',
       u'import_notes', u'iso3', u'max', u'min', u'name', u'region',
       u'region_ft_style', u'status', u'status_ft_style', u'stdDev',
       u'subregion', u'subregion_ft_style'],
      dtype='object')

In [123]:
serie_m['name'].head() # anche serie_m.name.head()

0            Zimbabwe
1              Kuwait
2             Algeria
3               Sudan
4    Hala'ib triangle
Name: name, dtype: object

In [124]:
serie_m['iso3'].head()

0    ZWE
1    KWT
2    DZA
3    SDN
4       
Name: iso3, dtype: object

In [125]:
serie_m['mean'].head() # non funziona con serie_m.mean.head() vedi sotto

0    1.108152
1    0.040666
2    0.688370
3    0.233098
4    0.065431
Name: mean, dtype: float64

In [126]:
serie_m.mean().head()

bbox_flag                   1.000000
claimed_by_ft_style         1.400000
fid                       145.632353
gaul_code                4417.014706
geometry_vertex_count    4745.147059
dtype: float64

In [127]:
df_m = pd.DataFrame(serie_m[['fid', 'gaul_code', 'iso3', 'mean', 'name', 'region','subregion']])

In [130]:
df_mms = pd.DataFrame(serie_mms[['iso3', 'min', 'max','stdDev']])

In [133]:
df_stats = df_m.join(df_mms, lsuffix='_df_m', rsuffix='_df_mms')

In [139]:
df_stats

,fid,gaul_code,iso3_df_m,mean,name,region,subregion,iso3_df_mms,min,max,stdDev
0,266.0,271.0,ZWE,1.108152,Zimbabwe,Africa,Eastern Africa,ZWE,0.002968,8.529884,0.482435
1,135.0,137.0,KWT,0.040666,Kuwait,Asia,Western Asia,KWT,0.000000,1.462511,0.069158
2,4.0,4.0,DZA,0.688370,Algeria,Africa,Northern Africa,DZA,0.000000,9.079371,0.461095
3,6.0,6.0,SDN,0.233098,Sudan,Africa,Northern Africa,SDN,0.000000,3.035463,0.135117
4,270.0,40760.0,,0.065431,Hala'ib triangle,Africa,Northern Africa,,0.000000,0.419073,0.042389
5,140.0,142.0,LSO,1.920143,Lesotho,Africa,Southern Africa,LSO,0.006115,28.568085,1.439830
6,8.0,8.0,AGO,0.987430,Angola,Africa,Middle Africa,AGO,0.000000,7.567527,0.395585
7,272.0,40765.0,EGY,0.377685,Egypt,Africa,Northern Africa,EGY,0.000000,5.257825,0.226465
8,141.0,144.0,LBR,0.975488,Liberia,Africa,Western Africa,LBR,0.007043,3.106822,0.278617
9,142.0,145.0,LBY,0.480061,Libya,Africa,Northern Africa,LBY,0.000000,4.904261,0.392254


In [136]:
# df_stats.to_csv('out.csv')